<a href="https://colab.research.google.com/github/IbramMedhat/Human-Motion-Synthesis/blob/main/Glocal_Net_Reimplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, PReLU, LeakyReLU, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU, Conv2D, Flatten, Reshape, Conv1D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Conv2DTranspose, Dense, BatchNormalization
from tensorflow.keras.datasets import mnist, cifar10
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import pandas as pd


# torch.manual_seed(0) # Set for testing purposes, please do not change!

print(tf.__version__)

2.8.0


In [ ]:
direction_csv_file = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/H3.6M/Directions 1.54138969.csv", header=None)

In [ ]:
direction_csv_file

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,473.6836,444.9424,500.9961,448.0299,479.8393,530.7856,506.2184,622.5688,493.6608,621.9954,...,515.4715,456.4298,515.4715,456.4298,499.2511,448.2281,515.0607,479.1209,515.0607,479.1209
1,473.6876,444.9526,500.9847,448.0307,479.8167,530.7855,506.2182,622.5726,493.6592,621.9958,...,514.9991,456.2006,514.9991,456.2006,498.6736,447.8242,514.2874,478.6892,514.2874,478.6892
2,473.6931,444.9663,500.9775,448.0403,479.7976,530.7957,506.2265,622.5868,493.6538,621.9990,...,514.7042,456.0592,514.7042,456.0592,498.3504,447.4883,513.7126,478.3876,513.7126,478.3876
3,473.7039,444.9854,500.9819,448.0659,479.7655,530.8015,506.2534,622.5867,493.6528,622.0079,...,514.5616,455.9726,514.5616,455.9726,498.1750,447.2817,513.3622,478.2127,513.3622,478.2127
4,473.7261,445.0067,501.0016,448.0924,479.7466,530.8076,506.2579,622.5981,493.6506,622.0058,...,514.6009,455.9780,514.6009,455.9780,498.1767,447.2362,513.2590,478.2043,513.2590,478.2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,508.8683,435.9992,531.6415,432.5145,531.7281,525.6750,536.5560,619.7991,507.8583,621.3291,...,552.5535,450.4184,552.5535,450.4184,535.6805,447.9043,557.0486,475.3463,557.0486,475.3463
1379,508.8894,435.9756,531.7077,432.5069,531.7758,525.6630,536.5682,619.7924,507.8712,621.3207,...,551.9902,450.3370,551.9902,450.3370,535.2592,447.4315,556.0175,475.3270,556.0175,475.3270
1380,508.9090,435.9860,531.6987,432.5204,531.7960,525.6733,536.5767,619.8024,507.8655,621.3173,...,551.3666,450.2951,551.3666,450.2951,534.9061,446.8693,554.7869,475.3963,554.7869,475.3963
1381,508.9323,435.9888,531.6935,432.5264,531.8196,525.6773,536.5802,619.8096,507.8681,621.3137,...,550.6077,450.2511,550.6077,450.2511,534.4236,446.4702,553.4522,475.5750,553.4522,475.5750


In [ ]:
#Convert to numpy array
direction_data_array = np.asarray(direction_csv_file)

In [ ]:

class GloGen_Bidirectional_RNN_Encoder(Model_):
  def __init__(self, num_recurrent_neurons=200):
    super(GloGen_Bidirectional_RNN_Encoder, self).__init__()
    
    self.recurrent_layer = LSTM(num_recurrent_neurons)

  def call(self, input_x):
    output = self.recurrent_layer(np.expand_dims(input_x, 2))
    #output = tf.one_hot(tf.argmax(output, axis = 1), depth=3)
    return output

In [ ]:
#Not sure of the output shape from the glonet
class GloGen_Bidirectional_RNN_decoder(Model_):
  def __init__(self, num_recurrent_neurons=64):
    super(GloGen_Bidirectional_RNN_decoder, self).__init__()
    
    self.recurrent_layer = LSTM(num_recurrent_neurons)

  def call(self, encoder_output):
    glogen_output = self.recurrent_layer(np.expand_dims(encoder_output, 2))
    #output = tf.one_hot(tf.argmax(output, axis = 1), depth=3)
    return glogen_output

In [ ]:
class LocGen_Bidirectional_RNN_encoder(Model_) :
  def __init__(self, num_recurrent_neurons=200) :
    super(LocGen_Bidirectional_RNN_encoder, self).__init__()
    self.recurrent_layer = LSTM(num_recurrent_neurons)

  def call(self, interpolated_output) :
    predictions = self.recurrent_layer(interpolated_output)
    return predictions

In [ ]:
class LocGen_Bidirectional_RNN_decoder(Model_) :
  def __init__(self, num_recurrent_neurons=200) :
    super(LocGen_Bidirectional_RNN_decoder, self).__init__()
    self.recurrent_layer = LSTM(num_recurrent_neurons)

  def call(self, interpolated_output) :
    predictions = self.recurrent_layer(interpolated_output)
    return predictions

In [ ]:
glogen_bidirectional_RNN_Encoder = GloGen_Bidirectional_RNN_Encoder()

In [ ]:
#Should have diminsions 100x60x64 as batch_siZe x timesteps x features
direction_data_array.shape

(1383, 64)

In [ ]:
output = bidirectional_RNN_Encoder(direction_data_array[:10])

In [ ]:
output.shape

TensorShape([10, 200])

In [ ]:
output

<tf.Tensor: shape=(1383, 200), dtype=float32, numpy=
array([[-1.2457371e-04, -5.7021580e-15, -1.7918209e-12, ...,
         4.4316658e-11, -3.4944052e-03,  1.0000000e+00],
       [-1.2564659e-04, -5.8740816e-15, -1.8363674e-12, ...,
         4.5298494e-11, -3.4932343e-03,  1.0000000e+00],
       [-1.2633204e-04, -5.9972770e-15, -1.8681473e-12, ...,
         4.5974533e-11, -3.4922527e-03,  1.0000000e+00],
       ...,
       [-1.3354421e-04, -7.3788589e-15, -2.2185053e-12, ...,
         5.3492020e-11, -3.7759892e-03,  1.0000000e+00],
       [-1.3312697e-04, -7.2885821e-15, -2.1959645e-12, ...,
         5.3014173e-11, -3.7716290e-03,  1.0000000e+00],
       [-1.3250113e-04, -7.1814706e-15, -2.1691726e-12, ...,
         5.2448774e-11, -3.7678732e-03,  1.0000000e+00]], dtype=float32)>